In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import sys

In [2]:
Leagues = { 'Serie A': 'https://fbref.com/en/comps/11/2022-2023/2022-2023-Serie-A-Stats',
            'Premier League': 'https://fbref.com/en/comps/9/2022-2023/2022-2023-Premier-League-Stats',
            'La Liga': 'https://fbref.com/en/comps/12/2022-2023/2022-2023-La-Liga-Stats',
            'Bundesliga': 'https://fbref.com/en/comps/20/2022-2023/2022-2023-Bundesliga-Stats',
            'Ligue 1': 'https://fbref.com/en/comps/13/2022-2023/2022-2023-Ligue-1-Stats'}

In [3]:
def scrap_clubs_links(link):
    Clubs = {}
    response = requests.get(link)
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        table = soup.select('#results2022-2023111_overall tbody')
        if table:
            table_html = str(table[0])
            table_soup = BeautifulSoup(table_html, 'html.parser')
            clubs = table_soup.find_all('tr')
            for row in clubs:
                Club = row.find('td', {'data-stat': 'team'}).text.strip()
                link = 'https://fbref.com/' + row.find('a')['href']
                Clubs[Club] = link
        else:
            print("Table not found on the page.")
    else:
        print("Failed to retrieve data. Status code:", response.status_code)
    return Clubs

In [4]:
attr = {      'StandardStats': 'stats_standard_11',
              'Goalkeeping': 'stats_keeper_adv_11',
              'Shooting': 'stats_shooting_11',
              'Passing': 'stats_passing_11',
              'Pass_types': 'stats_passing_types_11',
              'GCA': 'stats_gca_11',
              'Defense': 'stats_defense_11',
              'Possession': 'stats_possession_11',
              'Playing_time': 'stats_playing_time_11',
              'Misc': 'stats_misc_11'
             }

In [5]:
def Scrap (table, club,keys,league):
    if table:
        table_html = str(table[0])
        table_soup = BeautifulSoup(table_html, 'html.parser')
        player_rows = table_soup.find_all('tr')
        player_data = []
        for row in player_rows:
            player_dict={}
            for key in keys:
                if key == 'club':
                    player_dict[key] = club
                elif key == 'league':
                    player_dict[key] = league
                elif key == 'player':
                    player_dict[key] = row.find('th', {'data-stat': key}).text.strip()
                elif key == 'nationality':
                    player_dict[key] = row.find('td', {'data-stat': key}).text.strip()[-3:]
                else:
                    player_dict[key] = row.find('td', {'data-stat': key}).text.strip()
            player_data.append(player_dict)
        df = pd.DataFrame(player_data)
    else:
        print("Table not found on the page.")
    return df

In [6]:
def Scrap_ds(response,j):
    soup = BeautifulSoup(response.text,'html.parser')
    table = soup.select(f'table#{j} tbody')
    table_html = str(table[0])
    table_soup = BeautifulSoup(table_html, 'html.parser')
    ds = table_soup.find_all('tr')
    data_stat_values = [tag.get('data-stat') for tag in ds[0].find_all(['td', 'th'])]
    data_stat_values.insert(4, 'club')
    data_stat_values.insert(5, 'league')
    data_stat_values.pop()
    return data_stat_values

In [7]:
url = Leagues['Serie A']
Clubs = scrap_clubs_links(Leagues['Serie A'])
data = {}
for i, j in attr.items():
    data[i] = []
cpt = 1
ds = {}
for club, link in Clubs.items():
    response = requests.get(link)
    if response.status_code == 200:
        for att, st_id in attr.items():
            if not(ds):
                for att, st_id in attr.items():
                    ds[att] = Scrap_ds(response,st_id)
            soup = BeautifulSoup(response.text, 'html.parser')
            table = soup.select(f'table#{st_id} tbody')
            data[att].append(Scrap(table,club,ds[att],'Serie A'))
        print(f'{cpt} - {club} data extracted successfully')
        cpt +=1
    else:
        print('status code : ', response.status_code)
        break

1 - Napoli data extracted successfully
2 - Lazio data extracted successfully
3 - Inter data extracted successfully
4 - Milan data extracted successfully
5 - Atalanta data extracted successfully
6 - Roma data extracted successfully
7 - Juventus data extracted successfully
8 - Fiorentina data extracted successfully
9 - Bologna data extracted successfully
10 - Torino data extracted successfully
11 - Monza data extracted successfully
12 - Udinese data extracted successfully
13 - Sassuolo data extracted successfully
14 - Empoli data extracted successfully
15 - Salernitana data extracted successfully
16 - Lecce data extracted successfully
17 - Spezia data extracted successfully
18 - Hellas Verona data extracted successfully
19 - Cremonese data extracted successfully
20 - Sampdoria data extracted successfully


In [8]:
data['StandardStats'][2]

,player,nationality,position,age,club,league,games,games_starts,minutes,minutes_90s,...,goals_per90,assists_per90,goals_assists_per90,goals_pens_per90,goals_assists_pens_per90,xg_per90,xg_assist_per90,xg_xg_assist_per90,npxg_per90,npxg_xg_assist_per90
0,Theo Hernández,FRA,DF,24,Milan,Serie A,32,32,"2,771",30.8,...,0.13,0.10,0.23,0.10,0.19,0.13,0.11,0.24,0.11,0.21
1,Fikayo Tomori,ENG,DF,24,Milan,Serie A,33,32,"2,760",30.7,...,0.03,0.03,0.07,0.03,0.07,0.04,0.03,0.07,0.04,0.07
2,Sandro Tonali,ITA,MF,22,Milan,Serie A,34,30,"2,713",30.1,...,0.07,0.23,0.30,0.07,0.30,0.08,0.19,0.27,0.08,0.27
3,Rafael Leão,POR,"FW,MF",23,Milan,Serie A,35,28,"2,421",26.9,...,0.56,0.30,0.86,0.56,0.86,0.40,0.26,0.66,0.40,0.66
4,Brahim Díaz,ESP,"MF,FW",22,Milan,Serie A,33,27,"1,925",21.4,...,0.28,0.33,0.61,0.28,0.61,0.28,0.26,0.54,0.28,0.54
5,Pierre Kalulu,FRA,DF,22,Milan,Serie A,34,26,"2,548",28.3,...,0.04,0.00,0.04,0.04,0.04,0.05,0.04,0.10,0.05,0.10
6,Olivier Giroud,FRA,FW,35,Milan,Serie A,33,25,"2,139",23.8,...,0.55,0.21,0.76,0.42,0.63,0.53,0.17,0.70,0.44,0.61
7,Ismaël Bennacer,ALG,MF,24,Milan,Serie A,28,24,"2,007",22.3,...,0.09,0.09,0.18,0.09,0.18,0.04,0.15,0.19,0.04,0.19
8,Mike Maignan,FRA,GK,27,Milan,Serie A,22,22,"1,978",22.0,...,0.00,0.05,0.05,0.00,0.05,0.00,0.00,0.00,0.00,0.00
9,Davide Calabria,ITA,DF,25,Milan,Serie A,25,21,"1,690",18.8,...,0.05,0.21,0.27,0.05,0.27,0.06,0.09,0.15,0.06,0.15


In [13]:
def get_size_in_mb(obj):
    bytes_size = sys.getsizeof(obj)
    return bytes_size / (1024 * 1024)  # convert bytes to megabytes

In [14]:
total_size_in_mb = sum(get_size_in_mb(df) for df in data.values())
print(f"The total size of the DataFrames in the dictionary is {total_size_in_mb} MB")


The total size of the DataFrames in the dictionary is 0.0023651123046875 MB


In [65]:
# Dictionary that will host the concatenated dataframes
total_data = {}

In [66]:
for i,j in data.items():
    total_data[i] = pd.concat(j, ignore_index=True)

In [78]:
# Checking for dupliates
duplicates = total_data['StandardStats']['player'].duplicated()

if duplicates.any():
    print("The column has duplicate values. They are:")
    print(total_data['StandardStats'][duplicates]['player'])
else:
    print("The column does not have any duplicate values.")

The column has duplicate values. They are:
336           Nicolò Rovella
385            Brandon Soppy
416    Giorgos Kyriakopoulos
418             Nadir Zortea
454            Nedim Bajrami
462          Emanuel Vignato
537            Youssef Maleh
574         Eldor Shomurodov
579         Szymon Żurkowski
619                Ivan Ilić
630          Roberto Piccoli
634            Antonín Barák
662            Marco Benassi
669         Felix Afena-Gyan
690             Bruno Amione
692            Bram Nuytinck
696              Sam Lammers
697         Francesco Caputo
700             Alex Ferrari
703           Ronaldo Vieira
704             Koray Günter
716            Fabio Depaoli
717           Emirhan İlkhan
Name: player, dtype: object


In [76]:
#These duplicates are for players' transfers during the season
total_data['StandardStats'][total_data['StandardStats']['player']=='Nicolò Rovella']

,player,nationality,position,age,club,league,games,games_starts,minutes,minutes_90s,...,goals_per90,assists_per90,goals_assists_per90,goals_pens_per90,goals_assists_pens_per90,xg_per90,xg_assist_per90,xg_xg_assist_per90,npxg_per90,npxg_xg_assist_per90
208,Nicolò Rovella,ITA,MF,20,Juventus,Serie A,3,0,27,0.3,...,0.00,0.00,0.00,0.00,0.00,0.30,0.00,0.30,0.30,0.30
336,Nicolò Rovella,ITA,MF,20,Monza,Serie A,25,21,"1,740",19.3,...,0.05,0.10,0.16,0.05,0.16,0.06,0.11,0.18,0.06,0.18


In [90]:
# Exporting the scrapped data into csv files
for i in total_data.keys():
    total_data[i].to_csv(i+'_SerieA_22_23.csv', index=False)

In [89]:
import os
import glob
#To empty the returned files

files = glob.glob('/kaggle/working/*')
for f in files:
    os.remove(f)